# Search Science

Search science is the name given to the empirical study of search engine behavior.  Typically we are interested in taking a scientific approach to improving a search engine.  In order to improve a search engine we need a hypothesis to motivate the proposed change and we need a methodology to evaluate the improvement we make.

The methodology is initially offline, if offline analysis looks promising then the change may be deployed online.

For example - we may be interested whether word stemming could improve the relevance of search results.  Before deployinga new search engine into production we would like to find a way to evaluate it offline.  For example comparing the performance of the search engine with and without stemming on an offline data set.  We know that this will not give us the same performance as we would observe online but it is assumed to be a good proxy and evidence to move to a more expensive online experiment.

The general search science process proceeds as follows:

*  Identify opportunities to improve the current algorithm - maybe by talking to customers, running employee bug bashing sessions or by analysing click behaviour on the site.  
*  Identify most promising change that could be implemented to address the main user concern identified in the previous step.
*  Implement the change to the search engine algorithm
*  Run offline analysis to determine is new approach beats current approach
*  Run online A/B test to confirm that that change is better in the real world.
*  If new approach is better launch, else go back to start.

How we determine if the new approach is better is a criticical an currently ambiguous point in this process.

In order to avoid being misled we need to be rigorous in our assessment of the new algorithm.  Offline analysis requires a set of queries - these queries need to represent a diverse yet representative set of queries for the search engine we are trying to optimise.  These queries will be the set that we use to optimise the search engine towards - so like when training a machine learning algorithm we need to make sure the changes we make generalise to any query - so we would avoid (for example) having queries that fell into a single topic.  

Any change to the search engine will almost certainly work better for one query - but may degrade performance for all other queries.  It is important to assess performance over the set of queries and not change the set in order to prove the new change is better.

Given a set of queries we need to look at what results the search engine returns and compare using metrics how the performance is better.  Since the goal is to generate more relevant results we need some form of ground truth that we can  use to determine relevant.  Ideally we want the most relevant documents (relative to the information need and user query) to be in the results set and when ranked to be at the top.

There are two types of metric we can use - those that consider the order of the ranked results and those that don't.  We will look at both here.  

We also haven't explained how we know if a specific document in the result set is relevant or not.  Again there are two ways we can determine relevance, specifically:

*  Human Judgement - take the top n serch results for our query set and ask humans to determin if the are relevant or not.  Advantage high quality data, not noisy, but expensive.
*  Behavioural Analysis - look at click events on the search page and use clicks as a proxy for relevance.  Advantage we get lots of data cheaply, but the data is typically noisy.

For this notebook we will be using data that has human judgement scores associated with it.

## Unordered Metrics

These metrics do not take into consideration the ranking order - just the relevant / not-relevant label.  As a result they are the same metrics we have seen when training machine learning models for supervised classification.

We start with a subset of queries Q and documents D.  These documents and queries are a small but representative subset of the queries and information needs we think our users have, as well as a small subset of the documents in the entire corpus (since we cannot afford to judge all documents).  For the subset of Q x D we have a relevance score.  Typically we have all queries in our golden set and then say the top 10 documents D returned by the algorithm. Documents not sent for human judgement will be given a relevance score of 0 and may or may not be included (check with Nadia).  We therefore have a data set like this:

Q  | Rank | D   | Relevance Score
---|------|-----|----------------
1  | 1    | 325 |   1
1  | 2    | 214 |   1
1  | 3    | 455 |   0
...| ...  | ... |   ...
1  | 9    | 372 |   1
1  | 10   | 232 |   0
2  | 1    | 894 |   1
2  | 2    | 118 |   1 
...| ...  | ... |   ...

The query Q maps to the query details - such as query terms and query features.  Similarly the document D maps to the document details - such a document terms and document feautres.  We may also have a map from the query and document that maps to Q-D features.

For offline evaluation this subset of queries and documents becomes our corpus.  We use this reduced corpus to evaluate different search engines. But importantly we __*only*__ test this corpus using the subset of queries - the golden set.

### Precision and Recall

For each algorithm A we can compute a contingency table (also known as confusion matrix) by considering the top k documents returned for each query.  By choosing k we are in effect choosing the point in the ranking above which we would like all documents to be relevant - this is like us prediciting that only the top k documents are relevant and the remainder are not relevant.  Once we do this we can look at the classification accuracy relative to the human judgement relevance score ground truth.

|                  || *Predicted*  | *Class*   ||
|------------------||--------------------------|||
|                  ||     -    |    +    | Total |
|*True*   |   -     |   TN     |    FP   |  N    |
|*Classs* |   +     |   FN     |    TP   |  P    |
|         | Total   |   N*     |    P*   |       |


Now we can define metrics based on the contingency table:

| Name                      |   Definition  |   Synonyms                                  |
|---------------------------|---------------|---------------------------------------------|
| False Positive Rate       |   FP / N      | Type-I error, 1-Specificity                 |
| True Positive Rate        |   TP / P      | 1-Type II error, power, sensitivity, recall |
| Positive Predictive Value |   TP / P*     | Precision, 1-false discovery proportion     |
| Negative Predictive Value |   TN / N*     |                                             |


Visual for this at https://en.wikipedia.org/wiki/Precision_and_recall - be careful not to introduce confusion here.  For the course we could just optimise on F1?

Sometimes we talk about the accuracy of a model as:

$
accuracy = \frac{(TP + TN)}{( N + P )}
$


Generally we are interested in precision and recall.  

* Precision
    * The fraction of cases predicted to be positive that are actually positive
    * High precision is acheived by ensuring that all the predicted positive cases are actually positive
    * Quality
* Recall
    * The fraction of the positive cases predicted to be positive 
    * High recall is acheived by a model predicts as many of the positive cases as possible 
    * Quantity

We can also define classification metrics that compbine both precision and recall.  The F1-score is one such metric that computes the geometric mean of precision and recall - applying equal emphasis to precision as recall:

$
F_1 = 2 \times \frac{ precision . recall}{precision + recall} 
$

But we may choose to attach emphasis to recall:

$
F_2 = 5 \times \frac{ precision . recall}{ (4 . precision) + recall} 
$

Or we may choose to place higher emphasis on precision:

$
F_{0.5} = 1.25 \times \frac{ precision . recall}{( 0.25 . precision) + recall} 
$


__IS THIS REALLY UNORDERED - I DON"T AGREE - since to construct contingency table we assume ranks__

For example, let's look at a single query:

Q  |   Rank   |  Relevance Score
---|----------|------------------
1  | 1  | 1
1  | 2  | 1
1  | 3  | 0
1  | 4  | 1
1  | 5  | 1
1  | 6  | 1
1  | 7  | 0
1  | 8  | 0
1  | 9  | 1
1  | 10 | 0

We can vary k from 1 through to 10 and for each k compute precision and recall.  For example if k = 1 then we have the following prediction and true class:

Q  |   k    |  Relevance Score | Predicted Score  
---|--------|------------------|-----------
1  | 1  | 1 | 1 
1  | 2  | 1 | 0 
1  | 3  | 0 | 0  
1  | 4  | 1 | 0  
1  | 5  | 1 | 0  
1  | 6  | 1 | 0  
1  | 7  | 0 | 0  
1  | 8  | 0 | 0  
1  | 9  | 1 | 0  
1  | 10 | 0 | 0  

Leading to this contingency table:

|                  || *Predicted*  | *Class*   ||
|------------------||--------------------------|||
|                  ||     -    |    +    | Total |
|*True*   |   -     |   TN=4     |    FP=0   |  N=4    |
|*Classs* |   +     |   FN=5     |    TP=1   |  P=6    |
|         | Total   |   N*=9    |    P*=1   |       |


And so precision at k = 1 is TP / P* = 1.00 and recall is TP / P = 1/6 = 0.16.

We can write a function to compute this for any k:

In [3]:
import numpy as np
relevance = np.array([1, 1, 0, 1, 1, 1, 0, 0, 1, 0])

# Can probably do this with masks

def pratk(truth):
    k = []
    p = []
    r = []
    for k in range (0:10):
        pred = np.
        
        
        

## Ordered Metrics
.  